In [2]:
import seaborn as sns
from random import seed
from random import randrange
from csv import reader
import numpy as np

df = sns.load_dataset('titanic')[["pclass", "sex", "age", "deck", "alone", "survived"]]
df["age"] = df["age"].fillna(np.mean(df["age"])).astype(int)
df["deck"] = df["deck"].astype(str).replace("nan", "O")

# make everything a string that should be a string
print(df.columns)
train = df[:-100].values
test = df[-100:].values


print(train)

Index(['pclass', 'sex', 'age', 'deck', 'alone', 'survived'], dtype='object')
[[3 'male' 22 'O' False 0]
 [1 'female' 38 'C' False 1]
 [3 'female' 26 'O' True 1]
 ...
 [3 'male' 1 'O' False 1]
 [1 'male' 46 'B' True 0]
 [3 'male' 29 'O' True 0]]


Leg uit hoe numpy slicing werkt. Leg simpele berekeningen uit.

count_nonzero


In [ ]:
def gini_impurity(labels):
  # bereken het percentage hoevaak elke class voorkomt in data
  p = [np.count_nonzero(labels == item) / len(labels) for item in set(labels)]  # bevat een lijst met een percentage voor elke class.

  # doe elk percentage in het kwadraat en tel dit allemaal bij elkaar op.
  pi = sum([percentage ** 2 for percentage in p])

  return 1 - pi # lager is beter

gini_impurity(train[:,-1])

In [ ]:
def make_split(values, labels, split_value):
    if type(split_value) is str:
        mask = values == val
    else:
        mask = values < val

    return values[mask], values[~mask], labels[mask], labels[~mask]

In [ ]:
def get_split(values, labels):

    options = np.unique(values)

    best_gini = 1
    best_value = None
    for val in options:
        if type(val) is str:
            mask = values == val
        else:
            mask = values < val

        left_labels = labels[mask]
        right_labels = labels[~mask]

        entropy_left = gini_impurity(left_labels)*(sum(mask)/len(labels)) 
        entropy_right = gini_impurity(right_labels)*(sum(~mask)/len(labels)) 

        candidate_gini = sum([entropy_left, entropy_right])

        if candidate_gini < best_gini:
            best_gini = candidate_gini
            best_value = val

    return best_value, best_gini

get_split(train[:,2], train[:,-1])  

In [ ]:
def get_best_split(train):
    best_gini = 1
    best_value = None
    best_feature_index = None
    for i in range(train.shape[-1] - 1):
        
        value, gini = get_split(train[:,i], train[:,-1])

        if gini < best_gini:
            best_value = value
            best_gini = gini
            best_feature_index = i

    if type(best_value) is str:
        mask = train[:,best_feature_index] == best_value
    else:
        mask = train[:,best_feature_index] < best_value

    left = train[mask]
    right = train[~mask]

    return left, right
    
a, b = get_best_split(train)

print(len(a), len(b))

In [ ]:
# leaf node
def leaf_node(train):
    return train[:,-1]


In [ ]:
# Build a decision tree
def grow_tree(train):
    left, right = get_best_split(train)
 
    # print(len(left), len(right))
    # if len(left) == 0 or len(right) == 0:
    #     print(left, right)
    #     leaf_node(left+right)

    print(len(left), len(right))

    if len(left) > 1:
        grow_tree(left)
    else:
        leaf_node(left)

    if len(right) > 1:
        grow_tree(right)
    else:
        leaf_node(right)

grow_tree(train)

In [ ]:
from random import seed
from random import randrange
from csv import reader
 
 
# Convert string column to float
def str_column_to_float(dataset, column):
 for row in dataset:
 row[column] = float(row[column].strip())
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
 correct = 0
 for i in range(len(actual)):
 if actual[i] == predicted[i]:
 correct += 1
 return correct / float(len(actual)) * 100.0
 
# Create a terminal node value
def to_terminal(group):
 outcomes = [row[-1] for row in group]
 return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
 left, right = node['groups']
 del(node['groups'])
 # check for a no split
 if not left or not right:
 node['left'] = node['right'] = to_terminal(left + right)
 return
 # check for max depth
 if depth >= max_depth:
 node['left'], node['right'] = to_terminal(left), to_terminal(right)
 return
 # process left child
 if len(left) <= min_size:
 node['left'] = to_terminal(left)
 else:
 node['left'] = get_split(left)
 split(node['left'], max_depth, min_size, depth+1)
 # process right child
 if len(right) <= min_size:
 node['right'] = to_terminal(right)
 else:
 node['right'] = get_split(right)
 split(node['right'], max_depth, min_size, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size):
 root = get_split(train)
 split(root, max_depth, min_size, 1)
 return root
 
# Make a prediction with a decision tree
def predict(node, row):
 if row[node['index']] < node['value']:
 if isinstance(node['left'], dict):
 return predict(node['left'], row)
 else:
 return node['left']
 else:
 if isinstance(node['right'], dict):
 return predict(node['right'], row)
 else:
 return node['right']
 
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
 tree = build_tree(train, max_depth, min_size)
 predictions = list()
 for row in test:
 prediction = predict(tree, row)
 predictions.append(prediction)
 return(predictions)
 
# Test CART on Bank Note dataset
seed(1)
# load and prepare data
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(len(dataset[0])):
 str_column_to_float(dataset, i)
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))